In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
%pip install --upgrade --quiet langchain-community langchain faiss-cpu sentence-transformers pypdf beautifulsoup4 langchain-huggingface langchain-openai -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/

In [3]:
from langchain.embeddings import SentenceTransformerEmbeddings

# Set model_kwargs with trust_remote_code=True
embeddings = SentenceTransformerEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1.5",
    model_kwargs={"trust_remote_code": True}
)

<ipython-input-3-f5f7964b1a13>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, PyPDFLoader

loader = PyPDFLoader("https://www.versusarthritis.org/media/24901/fibromyalgia-information-booklet-july2021.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
vector_store = FAISS.from_documents(docs, embeddings)

In [5]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [6]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = hub.pull("rlm/rag-prompt")

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("mohamedalcafory/PubMed_Llama3.1_Based_model")
model = AutoModelForCausalLM.from_pretrained("mohamedalcafory/PubMed_Llama3.1_Based_model")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [8]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
  

In [12]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

llm = HuggingFacePipeline(pipeline=pipe)

In [13]:
def format_docs(docs):
  return"\n\n".join(doc.page_content for doc in docs)

In [14]:
generator_chain = (
    RunnablePassthrough.assign(input=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
retriever = vector_store.as_retriever()

In [17]:
retrieve_docs = (lambda x: x["query"]) | retriever
rag_chain = RunnablePassthrough.assign(context=retrieve_docs).assign(answer=generator_chain)
response = rag_chain.invoke({"query": "how does Physiotherapy works with Fibromyalgia"})
print(response)

{'query': 'how does Physiotherapy works with Fibromyalgia', 'context': [Document(metadata={'source': 'https://www.versusarthritis.org/media/24901/fibromyalgia-information-booklet-july2021.pdf', 'page': 6}, page_content='12 13 versusarthritis.org\nHelpline 0800 5200 520Treatments\nThere are a number of ways your symptoms can be managed.\nIn many cases, fibromyalgia can be diagnosed and treated by your \nGP, without the need to see a specialist.\nThey can advise you on treatments and therapies that tackle specific \nsymptoms. These may include drug treatments. However, physical and psychological therapies can often be a more effective way of managing fibromyalgia than medication.\nThere are also many things you can do for yourself that can change \nthe way your condition impacts upon your life.\nPhysical therapies\nDepending on how your fibromyalgia is affecting you, your doctor may refer you to a physiotherapist or occupational therapist.\nPhysiotherapy\nPhysiotherapy can help relieve t

In [21]:
print(response["answer"])

how does Physiotherapy works with Fibromyalgia?
Fibromyalgia is a condition characterized by chronic pain and stiffness in the muscles, tendons, and joints. It can also affect other areas of your body, like skin rashes or sensitivity to light.
Physiotherapists are trained professionals who work with patients to help them move better and reduce their discomfort. They may use various techniques, including exercise therapy and manual therapies, such as massage and joint mobilization, along with education on how to manage symptoms at home.
In this article we will explain which physiotherapeutic interventions could be used for fibromyalgia treatment:
1. Exercise Therapy: Gentle exercises that strengthen core muscle groups can improve posture, balance, and walking ability. Patients learn specific stretching and strengthening programs tailored to their needs, which they perform regularly.
2. Manual Therapies: Massage therapy involves gentle strokes over affected areas to relax muscles and red

In [24]:
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

rag_chain = RunnablePassthrough.assign(context=retrieve_docs).assign(answer=generator_chain)


def chat_with_chain():
    print("Welcome to LangChain Chat with Llama 3.1!")
    print("Type 'quit' to exit")

    while True:
        user_input = input("\nYou: ")

        if user_input.lower() == 'quit':
            print("Goodbye!")
            break

        try:
            retrieve_docs = (lambda x: x["query"]) | retriever
            response = rag_chain.invoke({"query": user_input})
            print(f"\nAssistant: {response['answer']}")

        except Exception as e:
            print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    chat_with_chain()

Welcome to LangChain Chat with Llama 3.1!
Type 'quit' to exit

You: how does Physiotherapy works with Fibromyalgia

Assistant: how does Physiotherapy works with Fibromyalgia
Fibromyalgia is a condition characterized by widespread musculoskeletal pain accompanied by fatigue, sleep, memory, mood and cognitive functional disorders. The exact cause of fibromyalgia is unknown; however, it may be associated with abnormalities in the processing of sensory information within the brain.
Physiotherapists use various techniques to help people recover from injury or disease. In the case of fibromyalgia, physiotherapy can be beneficial in improving the symptoms, enhancing physical functioning, reducing stiffness and disability. However, physiotherapy needs to be tailored according to individual patient's need because every person experiences different types and degrees of symptomatology.
The main goals are:
1) To improve body awareness through education on posture, movement patterns and ergonomics 